In [1]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import requests
from dotenv import load_dotenv

load_dotenv()

d:\GRIND\GEN_AI\LangChain\env2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [18]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [19]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    '''
     This function fetches the currency conversion factor between a given base currency and target currency
    '''
    url = f"https://v6.exchangerate-api.com/v6/5468efe4266161b81cdbec99/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    data = response.json()
    return data

@tool
def convert_currency(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    '''
     given a currency rate this finction calculates the target value form a giver base currency value
    '''
    return base_currency_value * conversion_rate

In [20]:
get_conversion_factor.invoke({"base_currency": "USD", "target_currency": "INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1761523201,
 'time_last_update_utc': 'Mon, 27 Oct 2025 00:00:01 +0000',
 'time_next_update_unix': 1761609601,
 'time_next_update_utc': 'Tue, 28 Oct 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.9117}

In [21]:
convert_currency.invoke({"base_currency_value": 100, "conversion_rate": 87.9117})

8791.17

In [42]:
# tool binding

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # or another supported model name
    temperature=0.7,
    max_tokens=None,  # use default or set a limit
    timeout=None,
    max_retries=2
)

#tool binding
model_with_tools = model.bind_tools([get_conversion_factor, convert_currency])

In [48]:
messages = [HumanMessage('What is the conversion factor betwwen USD and INR, and convert 10 USD to INR')]

In [49]:
model_message = model_with_tools.invoke(messages)
messages.append(model_message)

In [50]:
import json
for tool_call in model_message.tool_calls:
    #execte the 1st tool and get the value  of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)
    
    if tool_call['name'] == 'convert_currency':
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert_currency.invoke(tool_call)
        messages.append(tool_message2)

In [51]:
messages

[HumanMessage(content='What is the conversion factor betwwen USD and INR, and convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert_currency', 'arguments': '{"base_currency_value": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ef0145c7-fd4e-422f-a236-98d34c1f736f-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '049f83ec-9394-410a-9e59-6e5733bc6848', 'type': 'tool_call'}, {'name': 'convert_currency', 'args': {'base_currency_value': 10.0, 'conversion_rate': 87.9117}, 'id': 'bbb911a5-365c-4f73-a56c-2e2152e3448b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 22, 'total_tokens': 110, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exc

In [53]:
model_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 87.9117. 10 USD is equivalent to 879.117 INR.'